In [ ]:
#Import essential libraries
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
#figure out a way to choose the right range of prices (probability distribution, etc)
def userInfo(feat):
  while True:  
    maxPrice = float(input("\nWhat maximum price are you willing to pay for your car? (ex: 30000) "))
    if maxPrice < 12000:
      print("\nThat's too low! $12,000 is the lowest we can do.")
    elif maxPrice > 2100000:
      print("\nThat's too high! $2,100,000 is the highest we can do.")
    else:
      feat = feat.loc[feat["MSRP"] <= maxPrice]
      if len(feat) > 5:
        price50 = np.percentile(feat["MSRP"], [50])
        feat = feat.loc[feat["MSRP"] >= float(price50)]
        feat.index = [i for i in range(len(feat))]
      break

  houseSize = "house" + input("\nIs the size of your household greater than 2? (y/n) ").upper()
  dailyCommute = "commute" + input("\nIs your daily commute less than 5 miles? (y/n) ").upper()
  ecoFriendly = input("\nDo you prefer eco-friendly cars? (y/n) ").upper()
  luxuryChoice = input("\nDo you prefer powerful luxurious cars? (y/n) ").upper()

  return feat, [houseSize, dailyCommute, ecoFriendly, luxuryChoice], maxPrice

In [ ]:
#took away 16 from reg and added 6 each to van and truck and 2 each to sport ad st wagon
stylePercents = {"houseY": {"regular car": 33, "van": 39, "truck": 17, "sport car": 2, "station wagon": 8},
                "houseN": {"regular car": 55, "van": 15, "truck": 21, "sport car": 5, "station wagon": 4},
                "commuteY": {"regular car": 55, "van": 20, "truck": 17, "sport car": 4, "station wagon": 4},
                "commuteN": {"regular car": 44, "van": 23, "truck": 22, "sport car": 5, "station wagon": 6}}

fuelPercents = {"houseY": {"gasoline": 32, "electric": 27, "other": 41},
                "houseN": {"gasoline": 36, "electric": 27, "other": 37},
                "commuteY": {"gasoline": 28, "electric": 30, "other": 42},
                "commuteN": {"gasoline": 40, "electric": 25, "other": 35}}

sizePercents = {"houseY": {"midsize/large": 67, "compact": 33},
                "houseN": {"midsize/large": 56, "compact": 44},
                "commuteY": {"midsize/large": 62, "compact": 38},
                "commuteN": {"midsize/large": 57, "compact": 43}}


styleDict = {"regular car": ["Sedan", "Hatchback", "SUV"],
             "van": ["Minivan"],
             "truck": ["Pickup"],
             "sport car": ["Coupe", "Convertible"],
             "station wagon": ["Station Wagon"],}

fuelDict = {"gasoline": ["regular", "premium", "diesel"],
            "electric": ["electric"],
            "other": ["natural gas", "flex-fuel"]}

sizeDict = {"midsize/large": ["Midsize/Large"],
            "compact": ["Compact"]}


In [ ]:
def getCategory(inputStr, inputDict):
  for key, value in inputDict.items():
    for elem in value:
      if elem in inputStr:
        return key

In [ ]:
def probScore(prob):
  if np.random.randint(1, 101) <= prob:
    return 1
  else:
    return 0

In [ ]:
#Function to remove "N/A" values from a list
def removeNA(inputList):
    return [elem for elem in inputList if elem != "N/A"]

In [ ]:
hpList = list(map(float, removeNA(feat["Engine HP"])))
hp75, hp25 = np.percentile(hpList, [75]), np.percentile(hpList, [25])

highwayList = list(map(float, removeNA(feat["Highway MPG"])))
highway75, highway25 = np.percentile(highwayList, [75]), np.percentile(highwayList, [25])

cityList = list(map(float, removeNA(feat["City MPG"])))
city75, city25 = np.percentile(cityList, [75]), np.percentile(cityList, [25])

In [ ]:
ecoCoeff, ecoUnit = 0.15, 0.1
ecoWords = ["natural gas", "flex-fuel", "electric", "Hybrid"]

def ecoScore(carInfo):
  score, ecoCount = 0, 0

  for i in range(3):
    if isinstance(carInfo[3], str) and ecoWords[i] in carInfo[3]:
      ecoCount += 1
  
  if isinstance(carInfo[9], str) and ecoWords[3] in carInfo[9]:
    ecoCount += 1

  score = ecoCount * ecoCoeff

  if carInfo[4] != "N/A" and float(carInfo[4]) > hp75:
    score -= ecoUnit
  elif carInfo[4] != "N/A" and float(carInfo[4]) < hp25:
    score += ecoUnit

  if carInfo[5] != "N/A" and float(carInfo[5]) > 8:
    score -= ecoUnit
  elif carInfo[5] != "N/A" and float(carInfo[5]) <= 4:
    score += ecoUnit

  if float(carInfo[12]) > highway75:
    score += ecoUnit
  elif float(carInfo[12]) < highway25:
    score -= ecoUnit

  if float(carInfo[12]) > city75:
    score += ecoUnit
  elif float(carInfo[12]) < city25:
    score -= ecoUnit

  if "premium" in carInfo[3] and "flex-fuel" not in carInfo[3]:
    score -= ecoUnit

  return score

In [ ]:
luxuryCoeff, luxuryUnit = 0.15, 0.05
luxuryWords = ["Crossover", "Exotic", "High-Performance", "Luxury"]

def luxuryScore(carInfo):
  score, luxuryCount = 0, 0

  for i in range(len(luxuryWords)):
    if isinstance(carInfo[9], str) and luxuryWords[i] in carInfo[9]:
      luxuryCount += 1
  
  score = luxuryCount * luxuryCoeff

  if carInfo[4] != "N/A" and float(carInfo[4]) > hp75:
    score += luxuryUnit
  elif carInfo[4] != "N/A" and float(carInfo[4]) < hp25:
    score -= luxuryUnit

  if carInfo[5] != "N/A" and float(carInfo[5]) > 8:
    score += luxuryUnit
  elif carInfo[5] != "N/A" and float(carInfo[5]) <= 4:
    score -= luxuryUnit

  if float(carInfo[12]) > highway75:
    score -= luxuryUnit
  elif float(carInfo[12]) < highway25:
    score += luxuryUnit

  if float(carInfo[12]) > city75:
    score -= luxuryUnit
  elif float(carInfo[12]) < city25:
    score += luxuryUnit

  if "Convertible" in carInfo[11] or "Coupe" in carInfo[11]:
    score += luxuryUnit * 4

  if carInfo[8] != "N/A" and float(carInfo[8]) == 2:
    score += luxuryUnit * 2

  return score

In [ ]:
def scoreCar(carInfo):
  score = 0

  style = getCategory(carInfo[11], styleDict)
  fuel = getCategory(carInfo[3], fuelDict)
  size = getCategory(carInfo[10], sizeDict)

  for i in range(2):
    score += probScore(stylePercents[userInputs[i]][style])
    score += probScore(fuelPercents[userInputs[i]][fuel])
    score += probScore(sizePercents[userInputs[i]][size])

  score += carInfo[14] / 10000

  if carInfo[6] == "manual" or carInfo[6] == "automated manual":
    score *= 0.7

  if userInputs[2] == "Y":
    score += ecoScore(carInfo)

  if userInputs[3] == "Y":
   score += luxuryScore(carInfo)

  if carInfo[2] >= 2015:
    score *= 1.2

  if carInfo[15] >= maxPrice - 5000:
    score *= 1.3

  return score

In [ ]:
def scoreAll(feat):
  scoreIndexDict = {}

  for index in tqdm(feat.index):
    scoreIndexDict[scoreCar(list(feat.iloc[index]))] = index

  return scoreIndexDict

In [ ]:
feat = pd.read_csv("featuresData.csv", na_filter = False)
feat.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Drivetrain,Number of Doors,Market Category,Vehicle Size,Vehicle Style,Highway MPG,City MPG,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335,6,manual,rear wheel drive,2,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300,6,manual,rear wheel drive,2,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300,6,manual,rear wheel drive,2,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230,6,manual,rear wheel drive,2,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230,6,manual,rear wheel drive,2,Luxury,Compact,Convertible,28,18,3916,34500


In [ ]:
feat, userInputs, maxPrice = userInfo(feat)


What maximum price are you willing to pay for your car? (ex: 30000) 45000

Is the size of your household greater than 2? (y/n) y

Is your daily commute less than 5 miles? (y/n) y

Do you prefer eco-friendly cars? (y/n) y

Do you prefer powerful luxurious cars? (y/n) y


In [ ]:
scoreIndexDict = scoreAll(feat)
allScores = list(scoreIndexDict.keys())
allScores.sort(reverse = True)

normScores = list((np.array(allScores) - min(allScores)) / (max(allScores) - min(allScores)) * 10)

100%|██████████| 2859/2859 [00:00<00:00, 3388.41it/s]


In [ ]:
print("Top 5 Recommendations:")
for i in range(5):
  carList = list(feat.iloc[scoreIndexDict[allScores[i]]])
  print("\n" + str(i + 1) + ")\n" + carList[0] + " " + carList[1] + " " + str(carList[2]) + " " + carList[11])
  print("Score: " + str(normScores[i]))

Top 5 Recommendations:

1)
Audi Q5 2016 4 Door SUV
Score: 10.0

2)
Ford Explorer 2017 4 Door SUV
Score: 8.736670162367975

3)
Cadillac ATS 2015 Sedan
Score: 8.50742245390563

4)
Nissan Murano 2016 4 Door SUV
Score: 8.490225137223849

5)
BMW 4 Series Gran Coupe 2017 Sedan
Score: 8.401546799899538


In [ ]:
 print("Bottom 5 Recommendations:")
for i in range(5):
  carList = list(feat.iloc[scoreIndexDict[allScores[len(allScores) - 1 - i]]])
  print("\n" + str(i + 1) + ")\n" + carList[0] + " " + carList[1] + " " + str(carList[2]) + " " + carList[11])
  print("Score: " + str(normScores[len(normScores) - 1 - i]))

Bottom 5 Recommendations:

1)
Mitsubishi Lancer Evolution 2014 Sedan
Score: 0.0

2)
GMC Canyon 2016 Crew Cab Pickup
Score: 0.06265421506807033

3)
Suzuki Equator 2010 Crew Cab Pickup
Score: 0.06478231111230405

4)
Subaru WRX 2015 Sedan
Score: 0.0731221469613281

5)
Lincoln Mark LT 2008 Crew Cab Pickup
Score: 0.12038121677246431


In [ ]:
print("Top 5 Recommendations:")
for i in range(5):
  print("\n" + str(i + 1) + ")\n" + str(feat.iloc[scoreIndexDict[allScores[i]]]))
  print("Score: " + str(allScores[i]))

Top 5 Recommendations:

1)
Make                                        Audi
Model                                         Q5
Year                                        2016
Engine Fuel Type     premium unleaded (required)
Engine HP                                    220
Engine Cylinders                               4
Transmission Type                      automatic
Drivetrain                       all wheel drive
Number of Doors                                4
Market Category                 Crossover,Luxury
Vehicle Size                       Midsize/Large
Vehicle Style                         4 Door SUV
Highway MPG                                   28
City MPG                                      20
Popularity                                  3105
MSRP                                       40900
Name: 1797, dtype: object
Score: 10.312380000000001

2)
Make                                            Ford
Model                                       Explorer
Year                       

In [ ]:
print("Bottom 5 Recommendations:")
for i in range(5):
  print("\n" + str(i + 1) + ")\n" + str(feat.iloc[scoreIndexDict[allScores[len(allScores) - 1 - i]]]))
  print("Score: " + str(allScores[len(allScores) - 1 - i]))

Bottom 5 Recommendations:

1)
Make                               Mazda
Model                     B-Series Truck
Year                                2009
Engine Fuel Type        regular unleaded
Engine HP                            207
Engine Cylinders                       6
Transmission Type                 manual
Drivetrain              four wheel drive
Number of Doors                        4
Market Category                      N/A
Vehicle Size                     Compact
Vehicle Style        Extended Cab Pickup
Highway MPG                           19
City MPG                              15
Popularity                           586
MSRP                               22150
Name: 106, dtype: object
Score: -0.3884700000000001

2)
Make                           Pontiac
Model                               G6
Year                              2008
Engine Fuel Type      regular unleaded
Engine HP                          219
Engine Cylinders                     6
Transmission Type       